# **Лаб2_Энькаэр Уэркэнь_ ИУ5И-21М**

# **Задание**

1. Выбрать набор данных (датасет), содержащий категориальные и числовые признаки и пропуски в данных. Для выполнения следующих пунктов можно использовать несколько различных наборов данных (один для обработки пропусков, другой для категориальных признаков и т.д.) Просьба не использовать датасет, на котором данная задача решалась в лекции.
2. Для выбранного датасета (датасетов) на основе материалов лекций решить следующие задачи:

  * устранение пропусков в данных;
  * кодирование категориальных признаков;
  * нормализация числовых признаков.

# **Ход работы**

In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
# Загрузка датасета
df = pd.read_csv("Tree_Data.csv")
df.head()

,No,Plot,Subplot,Species,Light_ISF,Light_Cat,Core,Soil,Adult,Sterile,...,AMF,EMF,Phenolics,Lignin,NSC,Census,Time,Event,Harvest,Alive
0,126,1,C,Acer saccharum,0.106,Med,2017,Prunus serotina,I,Non-Sterile,...,22.00,NaN,-0.56,13.86,12.15,4,14.0,1.0,NaN,NaN
1,11,1,C,Quercus alba,0.106,Med,2017,Quercus rubra,970,Non-Sterile,...,15.82,31.07,5.19,20.52,19.29,33,115.5,0.0,NaN,X
2,12,1,C,Quercus rubra,0.106,Med,2017,Prunus serotina,J,Non-Sterile,...,24.45,28.19,3.36,24.74,15.01,18,63.0,1.0,NaN,NaN
3,2823,7,D,Acer saccharum,0.080,Med,2016,Prunus serotina,J,Non-Sterile,...,22.23,NaN,-0.71,14.29,12.36,4,14.0,1.0,NaN,NaN
4,5679,14,A,Acer saccharum,0.060,Low,2017,Prunus serotina,689,Non-Sterile,...,21.15,NaN,-0.58,10.85,11.20,4,14.0,1.0,NaN,NaN


# --- 1. Устранение пропусков ---
Удаляем признаки с большим количеством пропусков

In [34]:
print("Текущие пропуски:")
print(df.isnull().sum()[df.isnull().sum() > 0])

Текущие пропуски:
EMF        1500
Event         1
Harvest    2079
Alive      2292
dtype: int64


In [35]:
threshold = len(df) * 0.7
missing = df.isnull().sum()
df = df.drop(columns=missing[missing > (len(df) - threshold)].index)

Заполняем числовые пропуски медианой

In [36]:
if 'EMF' in df.columns:
    df['EMF'] = df['EMF'].fillna(df['EMF'].median())
if 'Event' in df.columns:
    df['Event'] = df['Event'].fillna(df['Event'].mode()[0])

Проверка пропусков после обработки

In [37]:
print("Оставшиеся пропуски:")
print(df.isnull().sum()[df.isnull().sum() > 0])

Оставшиеся пропуски:
Series([], dtype: int64)


# --- 2. Кодирование категориальных признаков ---

In [38]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

OneHotEncoder с удалением первой категории

In [40]:
encoder = ColumnTransformer([
    ("cat", OneHotEncoder(drop='first', sparse_output=False), categorical_cols)
], remainder='passthrough')

Применим преобразование

In [41]:
encoded = encoder.fit_transform(df)
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

In [42]:
print("\nПосле кодирования категориальных признаков:")
print("Размерность:", encoded_df.shape)
print("Первые строки:")
print(encoded_df.head())


После кодирования категориальных признаков:
Размерность: (2783, 85)
Первые строки:
   cat__Subplot_B  cat__Subplot_C  cat__Subplot_D  cat__Subplot_E  \
0             0.0             1.0             0.0             0.0   
1             0.0             1.0             0.0             0.0   
2             0.0             1.0             0.0             0.0   
3             0.0             0.0             1.0             0.0   
4             0.0             0.0             0.0             0.0   

   cat__Species_Prunus serotina  cat__Species_Quercus alba  \
0                           0.0                        0.0   
1                           0.0                        1.0   
2                           0.0                        0.0   
3                           0.0                        0.0   
4                           0.0                        0.0   

   cat__Species_Quercus rubra  cat__Light_Cat_Low  cat__Light_Cat_Med  \
0                         0.0                 0.0      

# --- 3. Нормализация числовых признаков ---

In [43]:
num_features_out = [col for col in encoded_df.columns if any(orig in col for orig in numeric_cols)]

scaler = StandardScaler()
encoded_df[num_features_out] = scaler.fit_transform(encoded_df[num_features_out])

Проверка результатов

In [44]:
print("\nНормализованные числовые признаки (первые строки):")
print(encoded_df[num_features_out].head())


Нормализованные числовые признаки (первые строки):
   remainder__No  remainder__Plot  remainder__Light_ISF  remainder__Core  \
0      -1.681460        -1.645604              0.791680         0.735510   
1      -1.732500        -1.645604              0.791680         0.735510   
2      -1.732056        -1.645604              0.791680         0.735510   
3      -0.484448        -0.492362             -0.222634        -1.359601   
4       0.783134         0.853087             -1.002875         0.735510   

   remainder__AMF  remainder__Phenolics  remainder__Lignin  remainder__NSC  \
0        0.117566             -1.265864          -0.280272       -0.481592   
1       -0.384572              1.653676           0.702262        1.179840   
2        0.316634              0.724501           1.324829        0.183911   
3        0.136254             -1.342026          -0.216835       -0.432726   
4        0.048502             -1.276019          -0.724330       -0.702651   

   remainder__Census  